In [0]:
dbutils.library.installPyPI("geopandas")
dbutils.library.installPyPI("shapely")
import sys
sys.version

Out[3]: '3.7.5 (default, Nov 7 2019, 10:50:52) \n[GCC 8.3.0]'

In [0]:
import pyspark
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, DateType, IntegerType, FloatType
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.utils import AnalysisException
import pyspark.sql.functions as F
import pandas as pd
from shapely.geometry import Point, Polygon, shape
from shapely import wkb, wkt
import geopandas as gpd
import numpy as np

In [0]:
import pyspark
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, DateType, IntegerType, FloatType
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.utils import AnalysisException
import pyspark.sql.functions as F
import pandas as pd
from shapely.geometry import Point, Polygon, shape
from shapely import wkb, wkt
import geopandas as gpd
import numpy as np


      
def transform_coma(algo):
  lista_final = []
  for h in algo:
    st_h = str(h)
    new_ok = st_h.replace(",", "").replace("[","").replace("]","")
    lista_final.append(new_ok)
  return lista_final


def wkt_loads(x):
    try:
        return wkt.loads(x)
    except Exception:
        return None

def primera_parte(data_file,json_file):  
  df_schema_1 = StructType() \
            .add(field= "VendorID", data_type=StringType(), nullable= True) \
            .add(field="tpep_pickup_datetime", data_type=DateType(), nullable= True) \
            .add(field="tpep_dropoff_datetime", data_type=DateType(), nullable= True) \
            .add(field="passenger_count", data_type=IntegerType(), nullable= True) \
            .add(field="trip_distance", data_type=DoubleType(), nullable= True) \
            .add(field="pickup_longitude", data_type=FloatType(), nullable= True) \
            .add(field="pickup_latitude", data_type=FloatType(), nullable= True) \
            .add(field="RatecodeID", data_type=IntegerType(), nullable= True) \
            .add(field="store_and_fwd_flag", data_type=StringType(), nullable= True) \
            .add(field="dropoff_longitude", data_type=FloatType(), nullable= True) \
            .add(field="dropoff_latitude", data_type=FloatType(), nullable= True) \
            .add(field="payment_type", data_type=IntegerType(), nullable= True) \
            .add(field="fare_amount", data_type=DoubleType(), nullable= True) \
            .add(field="extra", data_type=DoubleType(), nullable= True) \
            .add(field="mta_tax", data_type=DoubleType(), nullable= True) \
            .add(field="tip_amount", data_type=DoubleType(), nullable= True) \
            .add(field="tolls_amount", data_type=DoubleType(), nullable= True) \
            .add(field="improvement_surcharge", data_type=DoubleType(), nullable= True) \
            .add(field="total_amount", data_type=DoubleType(), nullable= True)
  
  df_coordinates_schema = StructType()\
                      .add(field = "zone_id", data_type=StringType(), nullable= True)\
                      .add(field = "zone_name", data_type=StringType(), nullable= True)\
                      .add(field = "geom", data_type=StringType(), nullable= True)
  
  df_with_schema = spark.read.format("csv") \
      .option("header", True) \
      .schema(df_schema_1) \
      .load(data_file)
  
  df_with_schema_col = df_with_schema.withColumn("year", year(df_with_schema.tpep_pickup_datetime)) \
            .withColumn("month", month(df_with_schema.tpep_pickup_datetime)) \
            .withColumn("day", dayofmonth(df_with_schema.tpep_pickup_datetime)) 
  
  #PARTE B
  data_json = spark.read.json(json_file)
  location_id = data_json.select("features.properties.locationid")
  segment_location_id = location_id.select("locationid").collect()
  lista_id = segment_location_id[0][0] #Lista de ID_Zonas extraidas del json
  lista_id_2 = [int(segment_location_id[0][0][i]) for i in range(len(lista_id))] 
  name_location = data_json.select("features.properties.zone")
  segment_name_location = name_location.select("zone").collect()
  lista_name = segment_name_location[0][0] #Lista de nombres extraidas del json 

  geometry_zone = data_json.select("features.geometry.coordinates")
  segment_geometry_zone = geometry_zone.select("coordinates").collect()
  
  geo_without_coma =[transform_coma(segment_geometry_zone[0][0][i][0][0]) for i in range(len(segment_geometry_zone[0][0]))]
  #geo_with_coma =[segment_geometry_zone[0][0][i][0][0] for i in range(len(segment_geometry_zone[0][0]))]
  lista_geometry_zone = ["Multipolygon " + str([[geo_without_coma[i]]]).replace("[","(").replace("]",")").replace("'","") for i in range(len(geo_without_coma))]
  
  id_name_coordinate_tuple = [(lista_id[i],lista_name[i],lista_geometry_zone[i]) for i in range(len(lista_name))] 
  pruebas = [wkt.loads(lista_geometry_zone[i]) for i in range(len(lista_geometry_zone))]
  
  #pruebas_2 = [{"type":"Multipolygon ", "coordinates":[geo_with_coma[i]]} for i in range(len(geo_without_coma))]
  #print(pruebas_2)
  
  data_id_zone_coordinates = {
  'zone_id':lista_id_2,
  'zone_name': lista_name,
  'geom': pruebas
  }

  #mk_sp_df = spark.createDataFrame(data=data_id_zone_coordinates, schema=df_coordinates_schema)
  df_id_zone_coordinates = pd.DataFrame(data_id_zone_coordinates, columns=['zone_id', 'zone_name', 'geom']) #Dataframe para utilizar con geopandas
  #df_id_zone_coordinates = df_id_zone_coordinates.assign(geom=pruebas)
  #df_id_zone_coordinates['geom']=df_id_zone_coordinates['geom'].apply(wkt.loads)
  gdf  = gpd.GeoDataFrame(df_id_zone_coordinates, geometry='geom')
  sc.broadcast(gdf)
  def name_zone(value_1, lista=id_name_coordinate_tuple):
    for i,j,k in lista:
      if(i==value_1):
        return j
    return 'Locacion NO registrada'

  def id_zone(value_1, lista=id_name_coordinate_tuple):
    for i,j,k in lista:
      if(j==value_1):
        return i
    return 'Locacion NO registrada'
  
  id_zone_UDF = F.udf(lambda x:id_zone(x), IntegerType())
  name_zone_UDF = F.udf(lambda x:name_zone(x))
  
  def find_borough(latitude, longitude): 
    mgdf = gdf.apply(lambda x: x['zone_name'] if x['geom'].intersects(Point(longitude,latitude)) else None, axis=1)
    idx = mgdf.first_valid_index()
    first_valid_value = mgdf.loc[idx] if idx is not None else None
    return first_valid_value
  
  find_borough_udf = F.udf(find_borough, StringType())
  
  df_raw_PU = df_with_schema_col.withColumn("pickup_zone", find_borough_udf(df_with_schema_col.pickup_latitude,df_with_schema_col.pickup_longitude))\
    .withColumn("dropoff_zone", find_borough_udf(df_with_schema_col.dropoff_latitude,df_with_schema_col.dropoff_longitude))
  df_final = df_raw_PU.withColumn("PULocationID", id_zone_UDF(df_raw_PU.pickup_zone))\
            .withColumn("DOLocationID", id_zone_UDF(df_raw_PU.dropoff_zone))
  
  return df_final
  

#"dbfs:/FileStore/tables/yellow_tripdata_2016_03.csv"
display(primera_parte("dbfs:/FileStore/tables/yellow_tripdata_2016_03.csv","/FileStore/tables/nyu_2451_36743_geojson.json"))

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,year,month,day,pickup_zone,dropoff_zone,PULocationID,DOLocationID
1,2016-03-01,2016-03-01,1,2.5,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.0,0.3,12.35,2016,3,1,Midtown North,East Chelsea,163,68
1,2016-03-01,2016-03-01,1,2.9,-73.98348,40.767925,1,N,-74.00594,40.733166,1,11.0,0.5,0.5,3.05,0.0,0.3,15.35,2016,3,1,Lincoln Square East,West Village,142,249
2,2016-03-01,2016-03-01,2,19.98,-73.78202,40.64481,1,N,-73.97454,40.67577,1,54.5,0.5,0.5,8.0,0.0,0.3,63.8,2016,3,1,JFK Airport,Park Slope,132,181
2,2016-03-01,2016-03-01,3,10.78,-73.86342,40.769814,1,N,-73.96965,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62,2016,3,1,LaGuardia Airport,Midtown East,138,162
2,2016-03-01,2016-03-01,5,30.43,-73.97174,40.792183,3,N,-74.17717,40.695053,1,98.0,0.0,0.0,0.0,15.5,0.3,113.8,2016,3,1,Upper West Side North,Newark Airport,238,1
2,2016-03-01,2016-03-01,5,5.92,-74.0172,40.705383,1,N,-73.97807,40.755787,1,23.5,1.0,0.5,5.06,0.0,0.3,30.36,2016,3,1,Battery Park City,Midtown Center,13,161
2,2016-03-01,2016-03-01,6,5.72,-73.99458,40.727848,1,N,0.0,0.0,2,23.0,0.5,0.5,0.0,0.0,0.3,24.3,2016,3,1,Greenwich Village South,null,114,null
1,2016-03-01,2016-03-01,1,6.2,-73.78877,40.64776,1,N,-73.82921,40.712345,3,20.5,0.5,0.5,0.0,0.0,0.3,21.8,2016,3,1,JFK Airport,Kew Gardens,132,134
1,2016-03-01,2016-03-01,1,0.7,-73.95822,40.76464,1,N,-73.967896,40.7629,1,5.5,0.5,0.5,2.0,0.0,0.3,8.8,2016,3,1,Lenox Hill East,Upper East Side South,140,237
2,2016-03-01,2016-03-01,3,7.18,-73.98578,40.74119,1,N,-73.94635,40.79788,1,23.5,0.5,0.5,3.2,0.0,0.3,28.0,2016,3,1,Union Sq,East Harlem North,234,74


In [0]:
#TAREA 2, PARTE 1 PUNTO A
#esctructura para datos que estan desde 2017 para delante
df_schema = StructType() \
            .add(field= "VendorID", data_type=StringType(), nullable= True) \
            .add(field="tpep_pickup_datetime", data_type=DateType(), nullable= True) \
            .add(field="tpep_dropoff_datetime", data_type=DateType(), nullable= True) \
            .add(field="passenger_count", data_type=IntegerType(), nullable= True) \
            .add(field="trip_distance", data_type=DoubleType(), nullable= True) \
            .add(field="RatecodeID", data_type=IntegerType(), nullable= True) \
            .add(field="store_and_fwd_flag", data_type=StringType(), nullable= True) \
            .add(field="PULocationID", data_type=IntegerType(), nullable= True) \
            .add(field="DOLocationID", data_type=IntegerType(), nullable= True) \
            .add(field="payment_type", data_type=IntegerType(), nullable= True) \
            .add(field="fare_amount", data_type=DoubleType(), nullable= True) \
            .add(field="extra", data_type=DoubleType(), nullable= True) \
            .add(field="mta_tax", data_type=DoubleType(), nullable= True) \
            .add(field="tip_amount", data_type=DoubleType(), nullable= True) \
            .add(field="tolls_amount", data_type=DoubleType(), nullable= True) \
            .add(field="improvement_surcharge", data_type=DoubleType(), nullable= True) \
            .add(field="total_amount", data_type=DoubleType(), nullable= True) \
            .add(field="congestion_surcharge", data_type=DoubleType(), nullable= True)

#eschema para datos de 2013 hasta 2017
df_schema_1 = StructType() \
            .add(field= "VendorID", data_type=StringType(), nullable= True) \
            .add(field="tpep_pickup_datetime", data_type=DateType(), nullable= True) \
            .add(field="tpep_dropoff_datetime", data_type=DateType(), nullable= True) \
            .add(field="passenger_count", data_type=IntegerType(), nullable= True) \
            .add(field="trip_distance", data_type=DoubleType(), nullable= True) \
            .add(field="pickup_longitude", data_type=FloatType(), nullable= True) \
            .add(field="pickup_latitude", data_type=FloatType(), nullable= True) \
            .add(field="RatecodeID", data_type=IntegerType(), nullable= True) \
            .add(field="store_and_fwd_flag", data_type=StringType(), nullable= True) \
            .add(field="dropoff_longitude", data_type=FloatType(), nullable= True) \
            .add(field="dropoff_latitude", data_type=FloatType(), nullable= True) \
            .add(field="payment_type", data_type=IntegerType(), nullable= True) \
            .add(field="fare_amount", data_type=DoubleType(), nullable= True) \
            .add(field="extra", data_type=DoubleType(), nullable= True) \
            .add(field="mta_tax", data_type=DoubleType(), nullable= True) \
            .add(field="tip_amount", data_type=DoubleType(), nullable= True) \
            .add(field="tolls_amount", data_type=DoubleType(), nullable= True) \
            .add(field="improvement_surcharge", data_type=DoubleType(), nullable= True) \
            .add(field="total_amount", data_type=DoubleType(), nullable= True)

def has_column(df, col):
    try:
        df[col]
        return True
    except AnalysisException:
        return False
            
file_location = "dbfs:/FileStore/tables/yellow_tripdata_2016_03.csv"

df_prueba = spark.read.format("csv") \
      .option("header", True) \
      .load(file_location)

if (has_column(df_prueba, "PULocationID")):
  
  print("primero")
  df_with_schema = spark.read.format("csv") \
      .option("header", True) \
      .schema(df_schema) \
      .load(file_location)
  df_with_schema_col = df_with_schema.withColumn("year", year(df_with_schema.tpep_pickup_datetime)) \
              .withColumn("month", month(df_with_schema.tpep_pickup_datetime)) \
              .withColumn("day", dayofmonth(df_with_schema.tpep_pickup_datetime)) 
			  
  display(df_with_schema_col)
else:
  print("segundo")
  df_with_schema = spark.read.format("csv") \
      .option("header", True) \
      .schema(df_schema_1) \
      .load(file_location)
  df_with_schema_col = df_with_schema.withColumn("year", year(df_with_schema.tpep_pickup_datetime)) \
              .withColumn("month", month(df_with_schema.tpep_pickup_datetime)) \
              .withColumn("day", dayofmonth(df_with_schema.tpep_pickup_datetime)) 
			  
  display(df_with_schema_col)
  
#display(df_with_schema_col.groupBy('PULocationID').count())

#pu_location_id = df_with_schema_col.select("PULocationID").collect()






segundo

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,year,month,day
1,2016-03-01,2016-03-01,1,2.5,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.0,0.3,12.35,2016,3,1
1,2016-03-01,2016-03-01,1,2.9,-73.98348,40.767925,1,N,-74.00594,40.733166,1,11.0,0.5,0.5,3.05,0.0,0.3,15.35,2016,3,1
2,2016-03-01,2016-03-01,2,19.98,-73.78202,40.64481,1,N,-73.97454,40.67577,1,54.5,0.5,0.5,8.0,0.0,0.3,63.8,2016,3,1
2,2016-03-01,2016-03-01,3,10.78,-73.86342,40.769814,1,N,-73.96965,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62,2016,3,1
2,2016-03-01,2016-03-01,5,30.43,-73.97174,40.792183,3,N,-74.17717,40.695053,1,98.0,0.0,0.0,0.0,15.5,0.3,113.8,2016,3,1
2,2016-03-01,2016-03-01,5,5.92,-74.0172,40.705383,1,N,-73.97807,40.755787,1,23.5,1.0,0.5,5.06,0.0,0.3,30.36,2016,3,1
2,2016-03-01,2016-03-01,6,5.72,-73.99458,40.727848,1,N,0.0,0.0,2,23.0,0.5,0.5,0.0,0.0,0.3,24.3,2016,3,1
1,2016-03-01,2016-03-01,1,6.2,-73.78877,40.64776,1,N,-73.82921,40.712345,3,20.5,0.5,0.5,0.0,0.0,0.3,21.8,2016,3,1
1,2016-03-01,2016-03-01,1,0.7,-73.95822,40.76464,1,N,-73.967896,40.7629,1,5.5,0.5,0.5,2.0,0.0,0.3,8.8,2016,3,1
2,2016-03-01,2016-03-01,3,7.18,-73.98578,40.74119,1,N,-73.94635,40.79788,1,23.5,0.5,0.5,3.2,0.0,0.3,28.0,2016,3,1


In [0]:
#PARTE 1 PUNTO B
file_location = "/FileStore/tables/nyu_2451_36743_geojson.json"
data = spark.read.json(file_location)
location_id = data.select("features.properties.locationid")
segment_location_id = location_id.select("locationid").collect()
lista_id = segment_location_id[0][0] #Lista de ID_Zonas extraidas del json
name_location = data.select("features.properties.zone")
segment_name_location = name_location.select("zone").collect()
lista_name = segment_name_location[0][0] #Lista de nombres extraidas del json 

geometry_zone = data.select("features.geometry.coordinates")
segment_geometry_zone = geometry_zone.select("coordinates").collect()

#geom = shape(pruebitas)
#print(geom.wkt)

def transform_coma(algo):
  lista_final = []
  for h in algo:
    st_h = str(h)
    new_ok = st_h.replace(",", "").replace("[","").replace("]","")
    lista_final.append(new_ok)
  return lista_final

#Arregral el jason para usar geopandas

#quitar las comas de dentro
geo_without_coma =[transform_coma(segment_geometry_zone[0][0][i][0][0]) for i in range(len(segment_geometry_zone[0][0]))]

#reemplazar brackets por parentesis
lista_geometry_zone = ["Multipolygon " + str([[geo_without_coma[i]]]).replace("[","(").replace("]",")").replace("'","") for i in range(len(geo_without_coma))]

id_name_coordinate_tuple = [(lista_id[i],lista_name[i],lista_geometry_zone[i]) for i in range(len(lista_name))] 

#aplicandole shapely a cada uno de los elementos de la lista
pruebas = [wkt.loads(lista_geometry_zone[i]) for i in range(len(lista_geometry_zone))]

data_id_zone_coordinates = {
  'id': lista_id,
  'zone_name': lista_name,
  'geom': pruebas
}
df_id_zone_coordinates = pd.DataFrame(data_id_zone_coordinates, columns=['id', 'zone_name', 'geom']) #Dataframe para utilizar con geopandas

def name_zone(value_1, lista = id_name_coordinate_tuple):
  for i,j,k in lista:
    if(i==value_1):
      return j
  return 'Locacion NO registrada'
  
def id_zone(value_1, lista = id_name_coordinate_tuple):
  for i,j,k in lista:
    if(j==value_1):
      return i
  return 'Locacion NO registrada'
  
id_zone_UDF = F.udf(lambda x:id_zone(x), IntegerType())
name_zone_UDF = F.udf(lambda x:name_zone(x))
  
  
if (has_column(df_with_schema_col, "PULocationID")):
  
  df_final = df_with_schema_col.withColumn('PULocationZone',name_zone_UDF(df_with_schema_col.PULocationID))\
                  .withColumn('DOLocationZone',name_zone_UDF(df_with_schema_col.DOLocationID))
  #display(df_final)

else:
  
  #df_id_zone_coordinates['geom']=df_id_zone_coordinates['geom'].apply(wkt.loads)
  gdf  = gpd.GeoDataFrame(df_id_zone_coordinates, geometry='geom')
    
  def find_borough(latitude, longitude): 
    mgdf = gdf.apply(lambda x: x['zone_name'] if x['geom'].intersects(Point(longitude,latitude)) else None, axis=1)
    idx = mgdf.first_valid_index()
    first_valid_value = mgdf.loc[idx] if idx is not None else None
    return first_valid_value
    
  find_borough_udf = F.udf(lambda x,y: find_borough(x,y))
  df_raw_PU = df_with_schema_col.withColumn("pickup_zone", find_borough_udf(df_with_schema_col.pickup_latitude,df_with_schema_col.pickup_longitude))\
    .withColumn("dropoff_zone", find_borough_udf(df_with_schema_col.dropoff_latitude,df_with_schema_col.dropoff_longitude))
    
    
  df_final = df_raw_PU.withColumn("PULocationID", id_zone_UDF(df_raw_PU.pickup_zone))\
            .withColumn("DOLocationID", id_zone_UDF(df_raw_PU.dropoff_zone))
  display(df_final)


In [0]:
#PARTE 2 PUNTO C 
pu_location_list = [row.PULocationID for row in df_final.select('PULocationID').collect()]
#pu_tuple = [(pu_location_list_id[i],pu_location_list_name) for i in range(len(pu_location_list_name))]
do_location_list = [row.DOLocationID for row in df_with_schema_col.select('DOLocationID').collect()]
matriz_OD_zeros = np.zeros([len(lista_id)+1,len(lista_name)+1
                           ])

for i in range(1, len(pu_location_list)):
  if (pu_location_list[i]<=263 and do_location_list[i]<=263):
    matriz_OD_zeros[pu_location_list[i]][do_location_list[i]] +=1 
    
    
data_correlacion = pd.DataFrame(matriz_OD_zeros)
display(data_correlacion)

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,426.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,

In [0]:
#PARTE 2 Punto D
frecuent_places = df_final.groupBy('DOLocationID').count().orderBy('count',ascending=False)
frecuent_places_names = frecuent_places.withColumn('NameLocation',name_zone_UDF(frecuent_places.DOLocationID)).take(10)
display(frecuent_places_names)
# agrupar por años tambien

DOLocationID,count,NameLocation
236,334323,Upper East Side North
237,296185,Upper East Side South
161,293782,Midtown Center
170,242037,Murray Hill
162,232451,Midtown East
230,225336,Times Sq/Theatre District
142,214164,Lincoln Square East
48,208624,Clinton East
234,204386,Union Sq
239,204350,Upper West Side South


In [0]:
#PARTE 2 PUNTO E
total_trip_distance = df_final.groupBy('year').agg({'trip_distance':'sum'})
display(total_trip_distance)

year,sum(trip_distance)
2003,0.0
2018,1259.68
2019,2.147663113999969E7
2009,163.26999999999998
2008,69.56
2001,0.0
2088,4.68


In [0]:
#PARTE 2 PUNTO G
# Cuanto se recauda anualmente con cada medio de pago
payement_type=df_final.groupBy('payment_type').agg({'total_amount':'sum'}).orderBy('sum(total_amount)',ascending=False)
display(payement_type)



payment_type,sum(total_amount)
1,9.187480686011899E7
2,2.7249980579893455E7
3,1025703.0800000741
4,97527.09999999913
